<a href="https://colab.research.google.com/github/OguzhanYildirim/Real-Time-Drone-and-Aircraft-Detection-/blob/main/Train_and_Test_YoloV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Jun 22 10:37:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15132, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 15132 (delta 26), reused 44 (delta 20), pack-reused 15073
Receiving objects: 100% (15132/15132), 13.50 MiB | 9.66 MiB/s, done.
Resolving deltas: 100% (10272/10272), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!echo "Drone" > /content/darknet/data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/darknet/backup' > data/obj.data
!mkdir /content/darknet/data/obj

mkdir: cannot create directory ‘/content/darknet/data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-06-22 10:40:12--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  19.8MB/s    in 8.0s    

2021-06-22 10:40:21 (19.4 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



In [ ]:
pip install -U --no-cache-dir gdown --pre

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.13.0-cp37-none-any.whl size=9046 sha256=c90c9c70a9f792e64988aaaf7c7a8b5c4d3926f19fbd1eafbbae021d2b1dd12a
  Stored in directory: /tmp/pip-ephem-wheel-cache-kitm3q12/wheels/ba/fa/c5/12813d7496f34652c43a471e11a780e769889d06e34735c32e
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4


In [ ]:
!gdown --id 1mpGDZ5-U_yKJETqLuxln0igrT9FhWsqN


Downloading...
From: https://drive.google.com/uc?id=1mpGDZ5-U_yKJETqLuxln0igrT9FhWsqN
To: /content/darknet/images.zip
64.5MB [00:01, 45.5MB/s]


**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip images.zip -d data/obj

Archive:  images.zip
replace data/obj/19th-july-2017-pahang-malaysia-260nw-680436289.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
0 0.317949 0.283929 0.271795 0.325000
['0', '0.317949', '0.283929', '0.271795', '0.325000']
1414 data/obj/annapolis-md-march-10-2018-260nw-1047939241.txt
0 0.317949 0.283929 0.271795 0.325000
0 0.288288 0.428571 0.387387 0.521429
['0', '0.288288', '0.428571', '0.387387', '0.521429']
1415 data/obj/istanbul-turkey-may-19-2015-260nw-280281050.txt
0 0.288288 0.428571 0.387387 0.521429
0 0.484615 0.446429 0.876923 0.735714
['0', '0.484615', '0.446429', '0.876923', '0.735714']
1416 data/obj/herceg-novi-montenegro-april-18-260nw-678682495.txt
0 0.484615 0.446429 0.876923 0.735714
0 0.505128 0.444643 0.143590 0.196429
['0', '0.505128', '0.444643', '0.143590', '0.196429']
1417 data/obj/udonthani-thailand-february-15-2017-260nw-582041197.txt
0 0.505128 0.444643 0.143590 0.196429
0 0.497436 0.460714 0.933333 0.642857
['0', '0.497436', '0.460714', '0.933333', '0.642857']
1418 data/obj/kuala-lumpur-malaysia-april-15-260nw-1068730016.txt
0 0.497436 0.46

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/krasnodar-russia-may-30-2017-260nw-1045738168.jpg', 'data/obj/hai-ukraine-october-20-2016-260nw-507979846.jpg', 'data/obj/pilsen-czech-republic-june-28-260nw-291803933.jpg', 'data/obj/hintersee-salzburger-landaustria-02092018-dji-260nw-1021785181.jpg', 'data/obj/dji-phantom-4-pro-obsidian-260nw-1088935289.jpg', 'data/obj/pilsen-czech-republic-july-07-260nw-673357708.jpg', 'data/obj/kagawa-japan-june-19-2016-260nw-439468954.jpg', 'data/obj/pancevo-serbia-03012018-dji-mavic-260nw-1038766843.jpg', 'data/obj/bangkok-thailand-june-18-2016-260nw-438990688.jpg', 'data/obj/dji-mavic-pro-flevoland-netherlands-260nw-1129344347.jpg', 'data/obj/kaunas-lithuania-march-03-2018-260nw-1038601063.jpg', 'data/obj/tuaran-sabahmalaysianov-262017-drone-dji-260nw-763582045.jpg', 'data/obj/bangkok-thailand-march-10-2018-260nw-1049162084.jpg', 'data/obj/russia-irkutsk-february-04-2017-260nw-573424861.jpg', 'data/obj/dji-mavic-pro-flevoland-netherlands-260nw-1129344416.jpg', 'data/obj/dji-mavic-air-

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

#Weighted model can upload here if u don't have time for a full run 


In [ ]:
%cd /content/darknet/backup
!gdown --id 1-6zewC-tPGrsXAKhK0jHzr1YUM6Tyi1G

/content/darknet/backup
Downloading...
From: https://drive.google.com/uc?id=1-6zewC-tPGrsXAKhK0jHzr1YUM6Tyi1G
To: /content/darknet/backup/yolov3_training_2000.weights
246MB [00:03, 65.4MB/s]


**6) Start the training**

In [ ]:
# Start the training
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

#!!!!! if u trained new model chance .weights file with /content/darknet/backup/yolov3_training_last.weights or .weights file name in def predictVideo function

In [ ]:
def predictVideo(videoDir):
  os.system(""" cd /content/darknet && ./darknet detector demo data/obj.data cfg/yolov3_training.cfg /content/darknet/backup/yolov3_training_2000.weights \
  -dont_show {} -i 0 -out_filename res.mp4
  """.format(videoDir))

In [ ]:
%cd /content/darknet
!gdown --id 1OolxkcIqGXmbH7qf0cM_O-cz2DEemyC3

/content/darknet
Downloading...
From: https://drive.google.com/uc?id=1OolxkcIqGXmbH7qf0cM_O-cz2DEemyC3
To: /content/darknet/drone_vid1.mp4
14.7MB [00:00, 41.0MB/s]


In [ ]:
predictVideo("/content/darknet/drone_vid1.mp4")

In [ ]:
from google.colab import files

files.download('/content/darknet/res.mp4')
